In [15]:
#Andrew Aquino
#DS681 - Deep Learning for Computer Vision
#Project Computer Using Agent

In [16]:
#This installs all the neccessary dependencies
!pip install -q gradio transformers accelerate bitsandbytes sentencepiece protobuf
!pip install -q pdf2image Pillow pymongo pydantic requests PyMuPDF
!pip install -q torch torchvision opencv-python-headless numpy
!pip install "pymongo[srv]"
!curl -fsSL https://ollama.com/install.sh | sh
!apt-get install -q -y poppler-utils

print(" Dependencies installed!")

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Reading package lists...
Building dependency tree...
Reading state information...
poppler-utils is already the newest version (22.02.0-2ubuntu0.12).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
 Dependencies installed!


In [17]:
#This Cell 2 starts the Ollama
import subprocess
import time
import requests
from threading import Thread
import pymongo
from pymongo import MongoClient

def start_ollama():
    subprocess.run(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

ollama_thread = Thread(target=start_ollama, daemon=True)
ollama_thread.start()
time.sleep(5)

print("Opening Llama model")
!ollama pull llama3.2:1b


Opening Llama model



In [18]:
#CELL 2.5 starts the MongoDB Connection using MongoDB Atlas
from google.colab import userdata
MONGO_URI = userdata.get('MONGO_URI')
client = MongoClient(MONGO_URI)

DB = client["cua_agent"]
COLLECTION = DB["paper_responses"]
COLLECTION.create_index("session_id")

print("Successfully connected to MongoDB Atlas.")

Successfully connected to MongoDB Atlas.


In [19]:
#Cell 3: Import Libraries and Core Setup
import gradio as gr
import uuid
import logging
from datetime import datetime
from typing import List, Dict, Any, Optional
import xml.etree.ElementTree as ET
import urllib.parse
import gc
import json
from io import BytesIO
from PIL import Image
import fitz
import cv2
import numpy as np
import threading

from pydantic import BaseModel, Field
from pdf2image import convert_from_bytes
import torch
from transformers import AutoProcessor, Qwen3VLForConditionalGeneration, BitsAndBytesConfig
# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Defining Global Variables for the VLM
VLM_PROCESSOR = None
VLM_MODEL = None
CURRENT_SESSION = str(uuid.uuid4())

In [20]:
#Cell 3.5: ways to organize the pdf files
PAPER_STORAGE = {
    "current_paper": { "pages": [], "loaded": False, "filename": None },
    "lock": threading.Lock()
}

#storing the pdf paper being analyzed
def store_paper(pages_data, filename):
    with PAPER_STORAGE["lock"]: PAPER_STORAGE["current_paper"] = { "pages": pages_data, "loaded": True, "filename": filename }
    logger.info(f"Stored paper: {filename}, {len(pages_data)} pages")

#function to get the paper
def get_paper():
    with PAPER_STORAGE["lock"]:
        return PAPER_STORAGE["current_paper"].copy()

#to clear an saved paper in the cach
def clear_paper():
    with PAPER_STORAGE["lock"]: PAPER_STORAGE["current_paper"] = { "pages": [], "loaded": False, "filename": None }


In [21]:
# Cell 4: Models avaiable and intialize the Pydantic Agent

# Available models
AVAILABLE_VLM_MODELS = {
    "Qwen2-VL-7B (High Performance)": "Qwen/Qwen2-VL-7B-Instruct",
    "LLaVA 1.6 Mistral 7B (Fast)": "llava-hf/llava-v1.6-mistral-7b-hf",
    "BLIP-2 OPT 2.7B (Faster)": "Salesforce/blip2-opt-2.7b",
}

AVAILABLE_LLM_MODELS = {
    "Llama 3.2 1B": "llama3.2:1b",
    "Llama 3.2 3B": "llama3.2:3b",
}

# Current configuration
CONFIG = {
    "vlm_model": "llava-hf/llava-v1.6-mistral-7b-hf",
    "llm_model": "llama3.2:1b",
    "max_pages": 25,
    "dpi": 150,
    "auto_analyze": True
}

class PaperMetadata(BaseModel):
    title: str
    authors: List[str] = []
    citation_count: int = 0
    abstract: str = ""


In [22]:
# Cell 5: These set of functions will be used to extract the highlighted text
def detect_yellow_highlights(image_array):
    hsv = cv2.cvtColor(image_array, cv2.COLOR_RGB2HSV)

    # Yellow color range in HSV
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([35, 255, 255])

    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # create the bounding box
    highlights = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 50 and h > 20:  # this will help filter the noise
            highlights.append({"x": x, "y": y, "width": w, "height": h})

    return highlights

#this will get text from the highlighted region
def extract_text_from_region(pdf_page, bbox):
    x, y, w, h = bbox["x"], bbox["y"], bbox["width"], bbox["height"]

    # PyMuPDF uses (x0, y0, x1, y1) format
    rect = fitz.Rect(x, y, x + w, y + h)
    text = pdf_page.get_text("text", clip=rect)

    return text.strip()


In [23]:
# Cell 6: Loads the VLM after selecting the model

def load_vlm_model(model_id):
    global VLM_PROCESSOR, VLM_MODEL

    logger.info(f"Loading VLM: {model_id}")

    # Unload existing model
    if VLM_MODEL is not None:
        del VLM_MODEL, VLM_PROCESSOR
        gc.collect()
        torch.cuda.empty_cache()

    quantization_config = BitsAndBytesConfig( load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16, bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True )

    logger.info("Loading the selected VLM")
    VLM_PROCESSOR = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

    # This is for the Qwen2 model
    if "Qwen2-VL" in model_id or "Qwen/Qwen2" in model_id:
        from transformers import Qwen2VLForConditionalGeneration
        VLM_MODEL = Qwen2VLForConditionalGeneration.from_pretrained(
            model_id,
            quantization_config=quantization_config,
            device_map="auto",
            trust_remote_code=True,
            low_cpu_mem_usage=True,
            torch_dtype=torch.float16
        )
    else:
        # this if for the other two models LLaVA and Blip
        VLM_MODEL = AutoModelForVision2Seq.from_pretrained(
            model_id,
            quantization_config=quantization_config,
            device_map="auto",
            trust_remote_code=True,
            low_cpu_mem_usage=True,
            torch_dtype=torch.float16
        )

    gc.collect()
    torch.cuda.empty_cache() #make sure I not using to much VRAM

    logger.info("VLM loaded successfully")
    return True, f"Loaded {model_id}"

In [24]:
# Cell 7: These functions handle the pdf files

def process_pdf_with_vlm(pdf_bytes, max_pages=25):
    global VLM_PROCESSOR, VLM_MODEL

    if VLM_MODEL is None:
        return None, "Please load a VLM model first"

    #just want to make sure to cap the dpi for shorter processing time
    dpi = min(CONFIG["dpi"], 200)
    logger.info(f"Converting PDF at {dpi} DPI")

    # this will convert PDF to images
    images = convert_from_bytes(
        pdf_bytes,
        dpi=dpi,
        fmt='png',
        first_page=1,
        last_page=max_pages
    )

    logger.info(f"Converted {len(images)} pages.")
    pages_data = []

    for i, img in enumerate(images):
        page_num = i + 1
        logger.info(f"Processing page {page_num}/{len(images)}...")

        # Aggressive resizing to prevent memory issues
        max_dim = 1024
        original_size = img.size

        if max(img.size) > max_dim:
            ratio = max_dim / max(img.size)
            new_size = tuple(int(dim * ratio) for dim in img.size)
            img = img.resize(new_size, Image.Resampling.LANCZOS)
            logger.info(f"  Resized from {original_size} to {new_size}")

        # Detect yellow highlights
        img_array = np.array(img)
        highlights = detect_yellow_highlights(img_array)


        # If using Qwen2-VL (different input format)
        if "Qwen2-VL" in CONFIG["vlm_model"]:
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": img},
                        {"type": "text", "text": "Extract all text from this document page, maintaining structure. Include equations and tables."}
                    ]
                }
            ]
            text = VLM_PROCESSOR.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            inputs = VLM_PROCESSOR(text=[text], images=[img], return_tensors="pt").to("cuda")

        else:
            # If using LLaVA/BLIP format
            prompt = "Extract all text from this document page, maintaining structure. Include equations and tables."
            inputs = VLM_PROCESSOR(text=prompt, images=img, return_tensors="pt").to("cuda")

        with torch.no_grad():
            outputs = VLM_MODEL.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=False
            )

        text = VLM_PROCESSOR.decode(outputs[0], skip_special_tokens=True)

        # Clean up output
        if "Extract all text" in text:
            text = text.split("Extract all text")[-1].strip()

        logger.info(f" Extracted {len(text)} characters")

        # Store page data
        pages_data.append({
            "page": page_num,
            "text": text,
            "highlights": highlights,
            "has_yellow_highlights": len(highlights) > 0
        })

        # making sure to cleanup after each page
        del inputs, outputs, img, img_array
        gc.collect()
        torch.cuda.empty_cache()

    logger.info(f"Completed processing {len(pages_data)} pages")

    return pages_data, f"Processed {len(pages_data)} pages"

#this will help identify important sections
def analyze_paper_structure(pages_data):
    important_sections = []

    for page in pages_data:
        text = page["text"].lower()

        # score given for important sections
        importance_score = 0
        reasons = []

        if "abstract" in text:
            importance_score += 10
            reasons.append("Contains abstract")

        if "contribution" in text or "novel" in text:
            importance_score += 8
            reasons.append("Discusses contributions")

        if "result" in text or "experiment" in text:
            importance_score += 7
            reasons.append("Contains results/experiments")

        if "table" in text or "figure" in text:
            importance_score += 5
            reasons.append("References tables/figures")

        if "relevance" in text:
            importance_score += 9
            reasons.append("Relevant portions of the paper compared to other studies")

        if importance_score >= 7:
            important_sections.append({
                "page": page["page"],
                "score": importance_score,
                "reasons": reasons,
                "text_preview": page["text"][:200] + "..."
            })

    return important_sections


In [25]:
# Cell 8: The LLM logic for questions and answering

#to make sure ollama is running correctly
def check_ollama_health():
    r = requests.get("http://localhost:11434/api/tags", timeout=3)
    return r.status_code == 200


def ensure_ollama_running():
    if check_ollama_health():
        return True

    logger.warning("Ollama not responding, attempting restart")


    # Kill existing processes
    os.system("pkill -9 ollama")
    time.sleep(2)

    # Restart
    subprocess.Popen(
        ["nohup", "ollama", "serve"],
        stdout=open('/tmp/ollama.log', 'a'),
        stderr=subprocess.STDOUT,
        preexec_fn=os.setpgrp
    )

    time.sleep(10)

    if check_ollama_health():
        logger.info("Ollama restarted successfully and is running")
        return True
    else:
        logger.error("Ollama restart failed")
        return False

#this for the llm answer required questions regarding the pdf file
def answer_question(question, pages_data, question_type="general"):

    combined_text = "\n\n--- PAGE BREAK ---\n\n".join( f"[Page {p['page']}]\n{p['text']}" for p in pages_data )

    # making customized prompts based on question type
    #this case is for yellow highlights
    if question_type == "yellow_highlights":
        highlighted_pages = [p for p in pages_data if p["has_yellow_highlights"]]
        if not highlighted_pages:
            return "No yellow highlights detected in the document."

        combined_text = "\n\n".join(
            f"[Page {p['page']} - Yellow Highlighted Text]\n{p['text']}"
            for p in highlighted_pages
        )

        prompt = f""" You are a helpful tutor. The user has highlighted text in yellow.

        Highlighted content from the paper: {combined_text}

        Question: {question}

        Provide a clear, tutorial-style explanation of the highlighted content. Break down complex concepts step-by-step."""

    #this case is for the important sections question
    elif question_type == "important_sections":
        important = analyze_paper_structure(pages_data)
        sections_text = "\n\n".join(
            f"[Page {s['page']}] (Importance: {s['score']}/10, Reasons: {', '.join(s['reasons'])})\n{s['text_preview']}"
            for s in important
        )

        prompt = f"""Identify and explain the most important sections in this paper.

        Important sections detected: {sections_text}

        Full paper text: {combined_text}

        Explain why these sections are important and what key information they contain."""

    #this case is for the relevance of the paper
    elif question_type == "relevance":
        prompt = f"""Analyze the relevance and originality of the study in this paper.

        Paper content: {combined_text}
        Question: {question}
        Focus on identifying relevance and originality of the study, how it relates to the world and its respective field of study."""

    else:
        prompt = f"""You are a research analyst. Answer based on the paper content.

        Paper text: {combined_text}
        Question: {question}
        Provide a detailed answer with page citations."""

    # just going to make sure to cut of the prompt if too long
    if len(prompt) > 80000:
        prompt = prompt[:80000] + "\n[...truncated...]"

    # Just to make Ollama is running before making request
    if not ensure_ollama_running():
        return "Ollama service is not available. Please restart the notebook or run Cell 2 again."

    # Call LLM with retry logic
    max_retries = 2
    for attempt in range(max_retries):
        try:
            logger.info(f"Sending request to Ollama (attempt {attempt + 1}/{max_retries})...")

            r = requests.post(
                "http://localhost:11434/api/generate",
                json={
                    "model": CONFIG["llm_model"],
                    "prompt": prompt,
                    "stream": False,
                    "options": {"temperature": 0.7, "num_predict": 2000}
                },
                timeout=600  # 10 minute timeout
            )

            answer = r.json()["response"]

            # Store interaction for the MondoDB
            COLLECTION.insert_one({
                "session_id": CURRENT_SESSION,
                "timestamp": datetime.now().isoformat(),
                "question": question,
                "question_type": question_type,
                "answer": answer,
                "pages_analyzed": len(pages_data)
            })

            return answer

        except Exception as e:
            return f" LLM Error: {str(e)}"


In [26]:
# Cell 9: This is the UI for the Gradio App

#The gradio app function to define the User Interface
def create_gradio_app():

    with gr.Blocks(title="CUA Resarch Paper Analyzer ", theme=gr.themes.Soft()) as app:

        gr.Markdown("# 📚 CUA Resarch Paper Analyzer 🤖")
        gr.Markdown("Upload PDFs, detect highlights, and ask questions about academic papers")

        with gr.Tabs():

            # This is for the first tab to choose the VLM, LLM and configuration setup
            with gr.Tab("VLM and LLM Configuration ⚙️"):
                gr.Markdown("### Model Configuration")

                with gr.Row():
                    vlm_dropdown = gr.Dropdown(
                        choices=list(AVAILABLE_VLM_MODELS.keys()),
                        value=list(AVAILABLE_VLM_MODELS.keys())[0],
                        label="Vision-Language Model (VLM)"
                    )

                    llm_dropdown = gr.Dropdown(
                        choices=list(AVAILABLE_LLM_MODELS.keys()),
                        value="Llama 3.2 1B",
                        label="Large Language Model (LLM)"
                    )

                load_vlm_btn = gr.Button("Load VLM Model", variant="primary")
                vlm_status = gr.Textbox(label="Status", interactive=False)

                gr.Markdown("Processing Options")

                with gr.Row():
                    max_pages_slider = gr.Slider(
                        minimum=1, maximum=30, value=15, step=1,
                        label="Max Pages to Process"
                    )

                    dpi_slider = gr.Slider(
                        minimum=75, maximum=200, value=150, step=25,
                        label="PDF DPI ( Higher DPI = Slower Proceessing Time )"
                    )


                save_config_btn = gr.Button("Save Configuration")
                config_status = gr.Textbox(label="Config Status", interactive=False)

                def update_config(vlm_name, llm_name, max_pages, dpi):
                    if dpi > 200:
                        return "DPI capped at 200 to prevent timeouts!"

                    CONFIG["vlm_model"] = AVAILABLE_VLM_MODELS[vlm_name]
                    CONFIG["llm_model"] = AVAILABLE_LLM_MODELS[llm_name]
                    CONFIG["max_pages"] = max_pages
                    CONFIG["dpi"] = min(dpi, 200)
                    return f"Configuration Saved: {CONFIG['vlm_model']}, {CONFIG["llm_model"]}"

                #this is for the buttons to load the VLM and LLM
                load_vlm_btn.click(
                    fn=lambda name: load_vlm_model(AVAILABLE_VLM_MODELS[name]),
                    inputs=[vlm_dropdown],
                    outputs=[vlm_status]
                )

                save_config_btn.click(
                    fn=update_config,
                    inputs=[vlm_dropdown, llm_dropdown, max_pages_slider, dpi_slider],
                    outputs=[config_status]
                )

            # this is the second tab to upload the paper and check the status of the LLM and MongoDB
            with gr.Tab("📄 PDF Paper Upload 🔍"):
                gr.Markdown("### Upload PDF Document")

                pdf_upload = gr.File(label="Upload PDF", file_types=[".pdf"])
                process_btn = gr.Button("Process PDF", variant="primary", size="lg")

                with gr.Row():
                    processing_status = gr.Textbox(label="Status", interactive=False)
                    pages_processed = gr.Number(label="Pages Processed", interactive=False)

                paper_info = gr.JSON(label="Paper Structure Analysis")

                #this is the process the pdf file that was uploaded
                def process_uploaded_pdf(pdf_file):
                    if pdf_file is None:
                        return "No file uploaded", 0, {}

                    # read PDF bytes
                    filename = pdf_file.name.split('/')[-1]
                    with open(pdf_file.name, "rb") as f:
                        pdf_bytes = f.read()

                    # process with the VLM
                    pages_data, status = process_pdf_with_vlm(pdf_bytes, CONFIG["max_pages"])

                    if pages_data is None:
                        clear_paper()
                        return status, 0, {}

                    store_paper(pages_data, filename)
                    important = analyze_paper_structure(pages_data)

                    #this is what the vlm will output after analyzing the paper
                    summary = {
                        "filename": filename,
                        "total_pages": len(pages_data),
                        "pages_with_highlights": sum(1 for p in pages_data if p["has_yellow_highlights"]),
                        "important_sections": len(important),
                        "important_details": important[:5]
                    }

                    return status, len(pages_data), summary

                process_btn.click(
                    fn=process_uploaded_pdf,
                    inputs=[pdf_upload],
                    outputs=[processing_status, pages_processed, paper_info]
                )

            #this is the third tab for the LLM questioning
            with gr.Tab("Ask Questions Regarding the Paper"):
                gr.Markdown("### Ask Questions About the Paper")


                with gr.Row():
                    check_services_btn = gr.Button("Ollama & MongoDB Status", size="sm")
                    service_status = gr.Textbox(label="Current Status", interactive=False, max_lines=2)
                    paper_loaded_status = gr.Textbox(label="Paper Status", interactive=False, max_lines=1)

                # To check if Ollama and MongoDB are running
                def check_services():
                    ollama_ok = check_ollama_health()

                    if client.server_info():
                        mongo_ok = True
                    else:
                        mongo_ok = False

                    status = []
                    status.append("Ollama is running" if ollama_ok else "Ollama not running")
                    status.append("MongoDB is connected" if mongo_ok else "MongoDB disconnected")

                    # Check paper status
                    paper = get_paper()
                    if paper["loaded"]:
                        paper_status = f"Paper loaded: {paper['filename']} ({len(paper['pages'])} pages)"
                    else:
                        paper_status = "Paper loaded unsuccessful"

                    return " | ".join(status), paper_status

                check_services_btn.click( fn=check_services, outputs=[service_status, paper_loaded_status] )

                question_type_radio = gr.Radio(
                    choices=[
                        "General Question",
                        "Explain Yellow Highlighted Text",
                        "Identify Important Sections",
                        "What is the Papers Relevance?"
                    ],
                    value="General Question",
                    label="Question Type"
                )

                question_input = gr.Textbox(
                    label="Your Question",
                    placeholder="ex: What is the main contribution of this paper?",
                    lines=3
                )

                ask_btn = gr.Button("Get Answer", variant="primary", size="lg")

                answer_output = gr.Textbox(
                    label="Answer",
                    lines=15,
                    interactive=False
                )

                # Some predefined questions
                gr.Markdown("### Quick Questions")

                with gr.Row():
                    q1_btn = gr.Button("Explain Yellow Highlighted Text")
                    q2_btn = gr.Button("Important Sections")
                    q3_btn = gr.Button("Papers Relevance")

                def handle_question(question, q_type):

                    # Get paper from previously defined functions
                    paper = get_paper()

                    if not paper["loaded"]:
                        return "Please upload and process a PDF first (Tab 2)"

                    pages_data = paper["pages"]

                    if not pages_data:
                        return "No pages found in the processed PDF"

                    type_map = {
                        "General Question": "general",
                        "Explain Yellow Highlighted Text": "yellow_highlights",
                        "Identify Important Sections": "important_sections",
                        "Papers Relevance": "relevance"
                    }

                    return answer_question(question, pages_data, type_map.get(q_type, "general"))

                # more buttons for easy questions
                ask_btn.click(
                    fn=handle_question,
                    inputs=[question_input, question_type_radio],
                    outputs=[answer_output]
                )

                q1_btn.click(
                    fn=lambda: handle_question("Explain the highlighted text", "Explain Yellow Highlights"),
                    outputs=[answer_output]
                )

                q2_btn.click(
                    fn=lambda: handle_question("What are the important sections?", "Identify Important Sections"),
                    outputs=[answer_output]
                )

                q3_btn.click(
                    fn=lambda: handle_question("What is the Papers Relevance?", "What is the Papers Relevance?"),
                    outputs=[answer_output]
                )
        return app


In [27]:
# Cell 10: This will Launch the Gradio App

# Create and launch the app
app = create_gradio_app()


print("Launching Gradion App")

#thius will also create a url to be opened in the browser tab
app.launch(
    share=True,
    server_name="0.0.0.0",
    server_port=7860,
    debug=True
)

/tmp/ipython-input-1800443061.py:6: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="CUA Resarch Paper Analyzer ", theme=gr.themes.Soft()) as app:


Launching Gradion App
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://275e4ca9faa9f3e026.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://275e4ca9faa9f3e026.gradio.live
